In [ ]:
import keras
from keras.models import Sequential
from keras.layers import *
import numpy as np

class value_predictor_by_image:
    def __init__(self, img_shape, weights_file = None, load_weights = False):
        self.img_shape = img_shape # shape of the input image
        self.weights_file = weights_file
        self.build_model()
        if load_weights:
            self.load_weights() # load saved weights
        self.compile()

    def build_model(self):
        self.model = Sequential([
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=self.img_shape),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(self.classNum, activation='relu'),
        ])

    def compile(self):
        self.model.compile(loss=keras.losses.mean_squared_error, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    # Predict one image's value
    def predict(self, data):
        data = self.format_array([data])
        return self.model.predict(data, batch_size = 1)[0]

    # Train the model
    def train(self, datas, targets, epochs = 100, batchSize = 32):
        datas = self.format_array(datas)
        targets = self.format_array(targets)
        self.model.fit(datas, targets, batch_size=batch_size, epochs=epochs, callbacks=None)

    # Reshape the images to the right format
    def format_array(self, array):
        return np.reshape(array, [len(array), *self.img_shape]).astype('float32')

    # Save weights
    def save_model(self):
        if self.weights_file != None:
            self.model.save_weights(self.weights_file)

    # Load weights
    def load_model(self):
        if self.weights_file != None and os.path.exists(self.weights_file):
            self.model.load_weights(self.weights_file)
            